# Dokumentation Medienverarbeitung Gruppe D 

### Allgemein
Die nachfolgende Dokumentation der Ergebnisse orientiert sich an dem **Cross Industry Standard Process for Data Mining** (kurz: CRISP DM), indem die Ergebnisse zu den jeweiligen Phasen beschrieben werden (Wirth & Hipp 2000). Abschließend wird ein Fazit des Projekts gezogen und weitere Forschungsmöglichkeiten aufgezeigt. Da im Laufe des Projekts zu viel Code entstanden ist, um die Implementierung und die Dokumentation in einem Notebook zu handhaben, wird die Dokumentation in diesem Notebook verfasst und auf die Implementierung in den zugehörigen Notebooks verwiesen.

# Business Understanding: 
#### Motivation/Usecase
Die Agrarwirtschaft ist und bleibt einer der wichtigsten Faktoren für das menschliche Leben und deren Erhaltung, da Pflanzen die Basis für die Ernährung liefern. Durch steigende Populationszahlen - und somit im Verhältnis immer weniger Anbaufläche pro Kopf - ist es von immer höherer Relevanz die Effizienz des Anbaus von Nutzpflanzen zu erhöhen. Vor allem Pflanzenkrankheiten können ein Problem darstellen und die Ernte stark beeinträchtigen. Daher ist es wichtig, eventuelle Krankheiten schnell zu erkennen und zu identifizieren, um den Schaden so gering wie möglich zu halten. Ziel dieses Projektes ist es daher ein Modell zu entwickeln, das anhand von Fotos und mittels Machine Learning sowohl die Spezies, als auch eventuelle Krankheiten auf Blättern erkennt und korrekt zuordnet. (Koerber, Kretschmer & Prinz 2008)

# Data Understanding: 
Damit ein solches Modell zuverlässige Ergebnisse liefert, muss zunächst eine möglichst große Menge Daten als Grundlage bereitgestellt werden. Dafür wurde zunächst das Git Repository [Plant Doc](https://github.com/pratikkayal/PlantDoc-Dataset) als Vorlage genutzt. Jedoch hat sich relativ schnell herausgestellt, dass die Bilder aus dem Repository nicht ausreichen um eine zuverlässige Pflanzenerkennung zu gewährleisten. 
Aus diesem Grund wurden lediglich ein die Label daraus übernommen und zusammen mit weiteren eigenen dazu verwendet, eine bessere Datenbasis zu schaffen. Hierfür wurde ein Image-Crawler für Bing verwendet (s. Notebook 'bing_image_crawler'), der automatisch das Netz nach vorher festgelegten Eingaben durchsucht und eine bestimmte Anzahl an Bildern herunterlädt. So konnte in kurzer Zeit eine erste Datenbasis aufgebaut werden, indem zu jedem der festgelegten Suchbegriffe 1500 Bilder runtergeladen wurden. Die Aufbereitung dieser Bilder ist in der Data Preparation beschrieben.

# Data Preparation 
### Aufbereiten und Labeln der gecrawlten Bilder: 
In diesem Schritt wurden die Bilder zunächst automatisiert aufbereitet. Dazu gehört das Löschen von Duplikaten sowie das entfernen von invaliden Bildern, die vom Programm nicht geöffnet oder verarbeitet werden können (s Notebook 'labeled_image_processing'). Daraufhin mussten die Bilder noch manuell gelabelt werden, wobei irrelevante und falsche Exemplare von Hand aussortiert wurden. Anschließend wurden die aussortierten Bilder von einer zweiten Person nochmals überprüft und ggf. bestätigt. Nach dem automatisierten Aussortieren und dem manuellen labeln sind ca. 2600 Bilder übergeblieben. 

#### Methoden zur Bildbearbeitung:
Im Zuge der Veranstaltung wurden verschiedene Methoden für die Bildverarbeitung gelehrt. Diese wurden auf den gecrawlten Daten mit angewendet, was in dem Notebook [image_preparation.ipynb](image_preparation.ipynb) implementiert ist. Hierbei war das Ziel, die Qualität der Bilder für das Trainig zu verbessern. Nachfolgend werden die angewendeten Methoden mit Beispielen dargestellt: 

- <b>Originales Bild</b>: 
<img src="./images/image_modification_original.png">

- <b>Histogramm-Equalisierung</b>: Bei der Histogramm-Equalisierung ist das Ziel, das ganze Farbspektrum auszunutzen, um so den Kontrast der Bilder zu verbessern. 
<img src="./images/image_modification_1.png">

- <b>Bild denoising</b>: Beim Image denoising soll Rauschen aus einem Bild entfernt werden, um den Fokus auf die relevanten Bereiche im Bild zu richten. 
<img src="./images/image_modification_2.png">

- <b>Anpassen der Helligkeit</b>: Die Anpassung der Helligkeit sollte helfen, Krankheitsbilder (z.B. rust leaf) deutlicher hervorzuheben.
<img src="./images/image_modification_3.png">
    
Diese Methoden wurden auf einer Stichprobe der Bilder durch Beaurteilung der Gruppe evaluiert. Insgesamt hat sich gezeigt, dass die relevanten Bereiche im Bild nicht verstärkt wurden. In mehreren Fällen haben sich die Bildmodifikationen kaum oder sogar negativ auf die Testbilder ausgewirkt (relevante Bereiche unscharf oder zu hell), sodass die Bilder für das finale Modell nicht mit den vorgestellten Methoden modifiziert wurden.  
#### Augmentierung: 
Nach der Akquise der Daten durch den Crawler und das automatisierte und manuelle Auussortieren der unpassenden Bilder sind ca. 2600 Bilder entstanden. Um die Anzahl der Trainingsdaten zu erhöhen und das Modell robuster gegenüber z.B. Rotationen zu machen, wurden die Bilder augmentiert. Dazu wurden folgende Methoden angewendet: 
     
    - Original:
<img src="./images/original_leaf.png">
    - Flip horizontal: Horizontale Spiegelung
<img src="./images/horizontalflip_leaf.png">
    - Flip vertikal: Vertikale Spiegelung
<img src="./images/vertflip_leaf.png">
    - Rotation: Das Bild wird um einen zufälligen Winkel zwischen - 90 und 90 Grad rotiert.
<img src="./images/rotation_leaf.png">
    
Somit wurden zu jedem Bild aus dem anfänglichen Datensatz 4 zusätzliche erzeugt (2 mal Rotation und jeweils 1 bei vertikalen und horizontalen Flip). Dies hat die Anzahl der Trainingsdaten von ca. 2600 auf etwa 12800 Bilder erhöht. Die Anzahl der Bilder pro Label ist nachfolgend dagestellt:   

Spezies:

    tomato     4206
    soybean    3327
    apple      1351
    peach      1312
    potato      836
    corn        758
    squash      535
    grape       475

Krankheiten:

    rust_leaf                     2896
    healthy                       2865
    leaf_blight                   2421
    tomato_leaf_spot              1301
    peach_leaf_bacterial_spot      675
    tomato_mold_leaf               600
    squash_powdery_mildew_leaf     505
    peach_leaf_curl                412
    tomato_leaf_mosaic_virus       355
    apple_scab_leaf                305
    tomato_leaf_yellow_virus       260
    grape_leaf_black_rot           205
    
#### Vorbereitung Daten:
Bevor das Modell nun mit dem Datensatz trainiert werden konnte wurden die Daten in ein [pandas](https://pandas.pydata.org) DataFrame übertragen, in dem jedes Bild mit seinem Pfad - sowie den zugehörigen Labels bezüglich der Spezies und der Krankheit - gespeichert wurde. In diesem Format können mit Hilfe des [ImageDataGenerators](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) die Bilder der Reihe nach an as Modell übergeben werden. Durch Angabe der Labels lernt das Modell, Spezies und Krankheiten zu klassifizieren.
Nach mehreren Durchläufen wurde bei einzelnen Spezies und Krankheiten deutlich, dass die geschaffene Datenbasis für diese nicht ausreichen würde, um zuverlässige Ergebnisse zu gewährleisten. Da es in diesem Projekt weniger um die Menge an unterschiedlichen Arten, als viel mehr um die Funktionalität im Allgemeinen geht, wurden einige dieser Spezies aus der Datenbasis entfernt.  

# Modelling: 
Im Verlauf des Projekts wurden verschiedene Arichtekturen und verschiedene Hyperparameter getestet und die Trainingsdaten angepasst, um das Modell iterativ zu verbessern. In den ersten Itertaionen ohne transfer learning, image augmentation und Anpassen der Trainingdaten wurde sowohl für die Klassifikation der Spezies als auch für die Klassifikation der Krankheit ein F1-Score von nur 54%-60% erreicht. Durch kontinuierliches Verbessern der Architektur und der Trainingsdaten konnte ein robustes Model entwickelt werden. Nachfolgend wird die Architektur der finalen Modelle beeschrieben: <br>

Wie in Abschnitt **Data Preparation** aufgezeigt, gibt es für diesen Anwendungsfall nur begrenzt viele Bilder, die qualitativ gut genug sind, um für das Trainig des Modells verwendet zu werden. Aus diesem Grund wurde transfer learning angewendet, da dies den Vorteil mitbringt, dass das vortrainierte Modell schon mit sehr vielen Bildern trainiert wurde. So können Layer für die Klassifikation an das vortrainierte Netz angehängt werden, um das Modell so auf diesen Anwendungsfall anzupassen und dann finetuning zu betreiben. Als vortrainiertes Model wurde [EfficientNet](https://github.com/qubvel/efficientnet) genutzt, da dieses eine state-of-the-art Performance bei relativ geringer Modellgröße liefert (Tan & Le 2019). <br>
### Architektur
Sowohl für die Klassifikation der Spezies als auch für die Klassifikation der Krankheit wurde das EfficientnetB0 mit den imagenet Gewichten genutzt und jeweils 3 Dense Layer mit 1024, 512 und Anzahl der Klassen Neuronen anghängt. Als optimizer wurde Adam und als loss wurde categorical crossentropy verwendet. <br>
Bei beiden Modellen wurden zuerst die angehängten Klassifikationslayer trainiert und anschließend alle Layer außer die ersten 100, um das mit imagenet trainierte Efficientnet noch besser auf diesen Anwenungsfall anzupassen. Um Overfitting zu verhindern wurde early stopping auf dem validation loss verwendet. So wird das Training abgebrochen, wenn zwei Epochen keine Verbesserung des validation loss festzustellen ist.


# Evaluation: 
### Klassifikation Spezies: 
Die Implementierung des finalen Models ist in dem Notebook [model_classification_species.ipynb](model_classification_species.ipynb) zu finden. Für die Klassifikation der Spezies wurde insgesamt 7 Epochen auf den anghängten Klassifikationslayern und 5 Epochen auf allen Layern außer den ersten 100 trainiert. Die Ergebnisse sind detailliert in den Tabelle 1 dargestellt. 

<br>
<center>Tabelle 1: Ergebnisse Modell Spezies</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{border-color:inherit;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-fymr{border-color:inherit;font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-c3ow"></th>
    <th class="tg-7btt">precision</th>
    <th class="tg-7btt">recall</th>
    <th class="tg-7btt">f1-score</th>
    <th class="tg-7btt">support</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-7btt">apple</td>
    <td class="tg-c3ow">0.93</td>
    <td class="tg-c3ow">0.90</td>
    <td class="tg-c3ow">0.91</td>
    <td class="tg-c3ow">282</td>
  </tr>
  <tr>
    <td class="tg-7btt">corn</td>
    <td class="tg-c3ow">0.99</td>
    <td class="tg-c3ow">0.93</td>
    <td class="tg-c3ow">0.96</td>
    <td class="tg-c3ow">163</td>
  </tr>
  <tr>
    <td class="tg-7btt">grape</td>
    <td class="tg-c3ow">0.93</td>
    <td class="tg-c3ow">0.94</td>
    <td class="tg-c3ow">0.93</td>
    <td class="tg-c3ow">95</td>
  </tr>
  <tr>
    <td class="tg-fymr">peach</td>
    <td class="tg-0pky">0.86</td>
    <td class="tg-0pky">0.89</td>
    <td class="tg-0pky">0.87</td>
    <td class="tg-0pky">300</td>
  </tr>
  <tr>
    <td class="tg-fymr">potato</td>
    <td class="tg-0pky">0.72</td>
    <td class="tg-0pky">0.83</td>
    <td class="tg-0pky">0.77</td>
    <td class="tg-0pky">191</td>
  </tr>
  <tr>
    <td class="tg-fymr">soybean</td>
    <td class="tg-0pky">0.97</td>
    <td class="tg-0pky">0.92</td>
    <td class="tg-0pky">0.94</td>
    <td class="tg-0pky">750</td>
  </tr>
  <tr>
    <td class="tg-fymr">squash</td>
    <td class="tg-0pky">0.98</td>
    <td class="tg-0pky">0.89</td>
    <td class="tg-0pky">0.94</td>
    <td class="tg-0pky">122</td>
  </tr>
  <tr>
    <td class="tg-fymr">tomato</td>
    <td class="tg-0pky">0.91</td>
    <td class="tg-0pky">0.95</td>
    <td class="tg-0pky">0.93</td>
    <td class="tg-0pky">897</td>
  </tr>
  <tr>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
  </tr>
  <tr>
    <td class="tg-fymr">accuracy</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky">0.92</td>
    <td class="tg-0pky">2800</td>
  </tr>
  <tr>
    <td class="tg-fymr">macro avg</td>
    <td class="tg-0pky">0.91</td>
    <td class="tg-0pky">0.91</td>
    <td class="tg-0pky">0.91</td>
    <td class="tg-0pky">2800</td>
  </tr>
  <tr>
    <td class="tg-fymr">weighted avg</td>
    <td class="tg-0pky">0.92</td>
    <td class="tg-0pky">0.92</td>
    <td class="tg-0pky">0.92</td>
    <td class="tg-0pky">2800</td>
  </tr>
  <tr>
    <td class="tg-fymr"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
</tbody>
</table>

Das Model erreicht einen gewichteten F1-Score von 92% auf den Validierungsdaten. Dabei sind die F1 Scores für alle Klassen bei über 90% außer bei potato (77%) und peach (83%). Dies ist rückblickend darauf zurückzuführen, dass beim manuellen labeln der Bilder einige schlechte Bilder nicht aussortiert wurden. Ein Beispiel für ein schlechtes Trainingsbild ist in Abbildung 1 dargestellt. Hier ist zu sehen, dass die gesamte Katroffelplanze auf dem Bild ist. Hier hätte ein Zuschnitt auf die einzelnen Blätter beim Labeln stattfinden müssen, wie in Abbildung 2 zu sehen ist. <br>
Insgesamt zeigt sich dennoch, dass das Modell insgesamt eine gute und robuste Klassifikation von Spezies ausweist.

<img src="./images/trainsample_schlecht.png" width="300" height="300">
<center>Abbildung 1: Beispiel Trainingsbild mit schlechter Datenqualität</center>

<img src="./images/trainsample_gut.png" width="300" height="400">
<center>Abbildung 2: Beispiel Trainingsbild mit gut Datenqualität</center>
<br>

### Klassifikation Krankheiten: 
Die Implementierung des finalen Models ist in dem Notebook [model_classification_diseases.ipynb](model_classification_diseases.ipynb) zu finden. Für die Klassifikation der Krankheiten wurde ebenfalls 7 Epochen auf den anghängten Klassifikationslayern und 5 Epochen auf allen Layern außer den ersten 100 trainiert. Dass die Anzahl der trainierten Epochen gleich ist, liegt nicht an festen Parametern für die Epochen. Das Zahl der Trainingsepochen wurde automatisch durch early stopping begrenzt. Die Ergebnisse auf den Validierungsdaten des Models sind detailliert in den Tabelle 2 dargestellt. 

<br>
<center>Tabelle 2: Ergebnisse Modell Krankheiten</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{border-color:inherit;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-c3ow"></th>
    <th class="tg-7btt">precision</th>
    <th class="tg-7btt">recall</th>
    <th class="tg-7btt">f1-score</th>
    <th class="tg-7btt">support</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-c3ow">apple_scab_leaf</td>
    <td class="tg-c3ow">0.98</td>
    <td class="tg-c3ow">0.91</td>
    <td class="tg-c3ow">0.94</td>
    <td class="tg-c3ow">46</td>
  </tr>
  <tr>
    <td class="tg-c3ow">grape_leaf_black_rot</td>
    <td class="tg-c3ow">0.89</td>
    <td class="tg-c3ow">0.97</td>
    <td class="tg-c3ow">0.93</td>
    <td class="tg-c3ow">34</td>
  </tr>
  <tr>
    <td class="tg-c3ow">healthy</td>
    <td class="tg-c3ow">0.90</td>
    <td class="tg-c3ow">0.88</td>
    <td class="tg-c3ow">0.89</td>
    <td class="tg-c3ow">521</td>
  </tr>
  <tr>
    <td class="tg-1wig">leaf_blight</td>
    <td class="tg-0lax">0.88</td>
    <td class="tg-0lax">0.79</td>
    <td class="tg-0lax">0.83</td>
    <td class="tg-0lax">427</td>
  </tr>
  <tr>
    <td class="tg-1wig">peach_leaf_bacterial_spot</td>
    <td class="tg-0lax">0.81</td>
    <td class="tg-0lax">0.77</td>
    <td class="tg-0lax">0.79</td>
    <td class="tg-0lax">127</td>
  </tr>
  <tr>
    <td class="tg-1wig">peach_leaf_curl</td>
    <td class="tg-0lax">0.93</td>
    <td class="tg-0lax">0.95</td>
    <td class="tg-0lax">0.94</td>
    <td class="tg-0lax">79</td>
  </tr>
  <tr>
    <td class="tg-1wig">rust_leaf</td>
    <td class="tg-0lax">0.88</td>
    <td class="tg-0lax">0.97</td>
    <td class="tg-0lax">0.93</td>
    <td class="tg-0lax">516</td>
  </tr>
  <tr>
    <td class="tg-1wig">squash_powdery_mildew_leaf</td>
    <td class="tg-0lax">0.97</td>
    <td class="tg-0lax">0.97</td>
    <td class="tg-0lax">0.97</td>
    <td class="tg-0lax">100</td>
  </tr>
  <tr>
    <td class="tg-1wig">tomato_leaf_spot</td>
    <td class="tg-0lax">0.82</td>
    <td class="tg-0lax">0.84</td>
    <td class="tg-0lax">0.83</td>
    <td class="tg-0lax">228</td>
  </tr>
  <tr>
    <td class="tg-1wig">tomato_mold_leaf</td>
    <td class="tg-0lax">0.90</td>
    <td class="tg-0lax">0.95</td>
    <td class="tg-0lax">0.93</td>
    <td class="tg-0lax">107</td>
  </tr>
  <tr>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
  </tr>
  <tr>
    <td class="tg-1wig">accuracy</td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax"></td>
    <td class="tg-0lax">0.89</td>
    <td class="tg-0lax">2185</td>
  </tr>
  <tr>
    <td class="tg-1wig">macro avg</td>
    <td class="tg-0lax">0.90</td>
    <td class="tg-0lax">0.90</td>
    <td class="tg-0lax">0.90</td>
    <td class="tg-0lax">2185</td>
  </tr>
  <tr>
    <td class="tg-1wig">weighted avg</td>
    <td class="tg-0lax">0.89</td>
    <td class="tg-0lax">0.89</td>
    <td class="tg-0lax">0.88</td>
    <td class="tg-0lax">2185</td>
  </tr>
</tbody>
</table>
 
Insgesamt erreicht das Model einen gewichteten F1-Score von 88%. Die Krankheiten tomato leaf spot (83%), peach leaf bacterial spot (79%) und leaf blight (83%) werden dabei schlechter erkannt als die anderen Krankheiten, welche einen F1-Score von 89%-97% aufweisen. 

### Visualisierung der Klassifikation
Um die Ergebnisse in der Evaluation besser nachvollziehen zu können, wurde der Einfluss von Bildbereichen für das Ergebnis der Klassifikation durch eine Heatmap visualiert. Die Implementierung ist in dem Notebook [heatmap_visualization.ipynb](heatmap_visualization.ipynb) zu finden. Diese Visualisierung ermöglicht es, nachvollziehen zu können, welche Teile eine Bildes starken bzw. schwachen Einfluss auf die Klassifikation haben. Dabei zeigen die eingefärbten Bereiche die einflussreichen Stellen, während die Farbe die stärke des Einflusses darstellt (rot: starker Einfluss). In den folgenden Beispielen (Abbildung 3 und 4) ist zu sehen, dass für die Klassifikation der Spezies die Kanten und die Struktur des Blatts stärker eingefärbt sind als der Rest des Bildes. Für die Klassifikation der Krankheiten ist der Bereich des Blattes stärker eingefärbt, in dem sich auch das Krankheitsbild abzeichnet.

<img src="./images/heatmap_species_gut_2.png" width="400" height="400"> 
<center>Abbildung 3: Beispiel passende Heatmap Klassifikation Spezies</center>

<img src="./images/heatmap_diseases_gut_2.png" width="400" height="400">
<center>Abbildung 4: Beispiel passende Heatmap Klassifikation Krankheit</center>

In anderen Beispielen sind die Hotspots der Heatmap weniger nachvollziehbar, wobei dann trotzdem oft die Klassifikation richtig ist. Ein Beispiel dafür ist in Abbildung XX zu sehen. Hier liegt ein großer Teil der einflussreichen Bereiche außerhalb des Blattes.

<img src="./images/heatmap_species_schlecht.png" width="400" height="400">
<center>Abbildung 5: Beispiel unpassende Heatmap Klassifikation Spezies</center>

<img src="./images/heatmap_diseases_schlecht.png" width="400" height="400">
<center>Abbildung 6: Beispiel unpassende Heatmap Klassifikation Krankheit</center>
<br>

Insgesamt zeigt sich, dass Model bei der Klassifikation der Spezies die Kanten der Blätter sowie die Struktur der Blätter einen starken Einfluss haben, während die Form des Blattes weniger ausschlaggeben zu sein scheint. Allerdings haben hier - trotz richtiger Klassifikation - öfter Bereiche eines Bildes einen starken Einfluss, die für den Menschen nicht intutiv nachvollziehbar sein. Bei der Klassifikation von Krankheiten scheint die Struktur des Blattes einen Einfluss auf das Ergebnis zu haben. Wenn kranke Stellen auf einem Bild zu erkennen sind, liegen die Hotspots der Heatmap sehr oft passend auf diesen Stellen des Blattes.  


# Deployment

### Aufbau der mobilen App
Die App wurde mit der IDE XCode und Programmiersprache Swift entwickelt. Die App verwendet nur einen ViewContoller um die UI anzuzeigen. Dabei wird ein CameraLayer über die aktuelle View gelegt. Der CameraLayer bekommt ein *delegate* Objekt gesetzt, damit der Camera Stream des Layers verwendet werden kann. Der Camera Stream liefert einen Pixelbuffer, auf welchem VisionML Requests durchgeführt werden.
Bei [Vision](https://developer.apple.com/documentation/vision) handelt es sich um ein Framework von Apple, mit dem AI-Modelle auf dem Gerät ausgeführt werden können. 
Das App Projekt befindet sich in dem Ordner [MyPlant-MobileApp](MyPlant-MobileApp) und lässt sich mit XCode auf einem Mac öffnen. Alternativ können die Swift Dateien auch mit einem beliebigen Texteditor geöffnet werden.

### Transfer der Modelle in die App
Für den Transfer der Tensorflow-Modelle in das CoreML Format, welches für iOS verwendbar ist, sollte das Tool [TF-CoreML](https://github.com/tf-coreml/tf-coreml) verwendet werden. Allerdings hat dieses aktuell Probleme mit der verwendeten Tensorflow Version aufgezeigt und war damit nicht direkt für anwendbar. Um trotzdem eine Visualisierung zu erreichen haben wir das Tool [Create ML](https://developer.apple.com/machine-learning/create-ml/) verwendet, welches mit der gleichen Datenbasis arbeiten konnte wie die Tensorflow Modelle. Wie in Abbildung 5 zu sehen ist, ist die Erstellung der Modelle mit CreateML jedoch nicht so effektiv wie mit Tensorflow und Python. Das lässt sich auch durch hohe Automation und die leichte Bedienbarkeit des Tools erklären, welche die Erstellung einfach, aber nicht flexibel anpassbar macht.
Die Genaugikeiten pro Spezies-Label finden sich in Abbildung 6 wieder.

<img src="./images/CreateML.png" width="800" height="400">
<center>Abbildung 5: CreateML Training der Pflanzen Klassifikation</center>
<br>
<img src="./images/CreateML2.png" width="800" height="400">
<center>Abbildung 6: CreateML Evaluation der Pflanzen Klassifikation</center>
<br>

Dafür können die beiden Modelle direkt in das XCode Projekt integriert und im Code verwendet werden. Da die Modelle an dieser Stelle neu trainiert werden, erhalten wir nicht das *Heatmap*-Feature wie in den Tensorflow Modellen, allerdings können der Pflanzenname und die Krankheit in Echtzeit in der App erkannt und angezeigt werden. Ein Beispiel Bildschirmfoto ist dazu in Abbilung 7 zu sehen

<img src="./images/Screenshot_App.png" width="300" height="600">
<center>Abbildung 7: Screenshot der Echtzeit-Erkennung in der mobilen iOS Applikation</center>
<br>

# Fazit des Projekts:
- wie viele Spezies und Krankheiten erkannt werden nennen
Die Evaluation der Modelle zeigt, dass eine Klassifikation der Spezies von Nutzpflanzen mit einem hohen F1-Score von 92% erreicht wurde. Ein ähnlich gutes Ergebnis zeigt sich für die Klassifikation der Krankheiten, bei der das Model ein F1-Score von 88% erreicht. Die F1-Scores der einzelnen Klassen liegen zwischen 77% und 97%, wobei ein großer Teil der Klassen über 90% und nur 2 Klassen unter 80% liegt. <br>

Um die Modelle testen zu können, sollten diese in einer mobilen App deployed werden. Da in diesem Projekt die Modelle nicht konvertiert werden konnten und daher das Heatmap-Feature, sowie die hohe Genauigkeit der EffitientNet Modelle nicht in die App integriert werden konnten, stellt dies einen Ausblick für die Zukunft dar.
Zusätzlich konnte die Erkennung in der App nur an Testbildern getestet werden und nicht an echten Nutzpflanzen.
In der Zukunft könnten die genaueren Tensorflow-Modelle jedoch konvertiert werden und in der mobilen App deployed werden. Zusammen mit zusätzlichen und besseren Trainingsdaten könnte die App sich dann auch für Landwirte mit Nutzpflanzen als sehr hilfreich erweisen. 

# Quellen: 

1. Wirth, R., & Hipp, J. (2000, April). CRISP-DM: Towards a standard process model for data mining. In Proceedings of the 4th international conference on the practical applications of knowledge discovery and data mining (Vol. 1). London, UK: Springer-Verlag.

2. Koerber, K., Kretschmer, J., & Prinz, S. (2008). Globale Ernährungsgewohnheiten und-trends. Asien, 2, 42

3. Tan, M., & Le, Q. (2019, May). Efficientnet: Rethinking model scaling for convolutional neural networks. In International Conference on Machine Learning (pp. 6105-6114). PMLR.